
# Example line noise filtering script

Filters the 60Hz line noise from the data, as well as the harmonics. Includes
environment checks for SLURM jobs for convenience


In [1]:
import mne
import os
from ieeg.mt_filter import line_filter
from ieeg.io import get_data, raw_from_layout, save_derivative
from ieeg import viz
from bids import BIDSLayout
from ieeg.viz.utils import figure_compare


In [2]:
HOME = os.path.expanduser("~")

In [3]:
if 'SLURM_ARRAY_TASK_ID' in os.environ.keys():
    LAB_root = os.path.join(HOME, "workspace", "CoganLab")
    subject = int(os.environ['SLURM_ARRAY_TASK_ID'])
else:  # if not then set box directory
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
    subject = 5

In [3]:
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")



# print(subjects)


subj = "D0057"
save_dir = os.path.join(layout.root, 'derivatives', 'spec', 'wavelet', subj)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)


raw = raw_from_layout(layout, subject=subj,
                        extension='.edf', preload=True)

filt = line_filter(raw, mt_bandwidth=10., n_jobs=6,
                filter_length='700ms', verbose=10,
                freqs=[60], notch_widths=20)


data = [raw, filt]
viz.utils.figure_compare(data, ["Un", ""], avg=True, n_jobs=6,
                verbose=10, proj=True, fmax=250)

save_derivative(filt, layout, "clean")

print("Data saved successfully.")





# # Save data to save_dir
# file_names = ["data_raw.fif", "data_filt.fif"]  # Specify the desired file names
# for d, file_name in zip(data, file_names):
#     file_path = os.path.join(save_dir, file_name)
#     d.save(file_path, overwrite=True)


# for sub in subjects:
#     # if sub != "D0022":
#     #     continue
#     # Load the data
#     raw = raw_from_layout(layout, subject=sub,
#                            extension='.edf', preload=True)
#     filt = line_filter(raw, mt_bandwidth=10., n_jobs=6,
#                    filter_length='700ms', verbose=10,
#                    freqs=[60], notch_widths=20)


#     data = [raw, filt]
#     viz.utils.figure_compare(data, ["Un", ""], avg=True, n_jobs=6,
#                    verbose=10, proj=True, fmax=250)

FileNotFoundError: Search terms matched more than one file: 
[<BIDSFile filename='C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf'>, <BIDSFile filename='C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_ieeg.edf'>] 
 try adding more search terms

save_derivative(filt, layout, "clean")

